# Sorting the conferences and HSF trainings via "date" field

In [ ]:
import json
import pandas as pd

## Sorting with pandas

In [ ]:
confs = pd.read_json("_data/Conference.json")
confs.dropna(inplace=True)
confs

In [ ]:
confs.sort_values("date", ascending=False)

In [ ]:
result = confs.sort_values("date", ascending=False).to_json(
    orient="records", date_format="iso"
)
parsed = json.loads(result)
parsed

In [ ]:
# writing to file called sorted_confs.json
json.dump(parsed, open("_data/sorted_confs.json", "w"), indent=4)

## Now for sorting the Trainings

In [ ]:
mentored = pd.read_json("_data/mentored.json")
mentored.drop_duplicates(keep=False, inplace=True)
mentored.sort_values("date", ascending=False, inplace=True)

In [ ]:
parsed = json.loads(mentored.to_json(orient="records", date_format="iso"))

json.dump(parsed, open("_data/sorted_mentored.json", "w"), indent=4)

# result = mentored.sort_values("date", ascending=False).to_json(
#     orient="records", date_format="iso"
# )
# # parsed = json.loads(result)
# # parsed

In [ ]:
# [{
#     "title": "HSF/IRIS-HEP Software Basics Training",
#     "location": "Virtual",
#     "date": "September 28-30, 2022",
#     "url": "https://indico.cern.ch/event/1190572/"
#   },
#   {
#     "title": "DANCE@Snowmass",
#     "location": "University of Washington, Seattle, USA",
#     "date": "July 19-23, 2022",
#     "url": "https://indico.cern.ch/event/1151329/"
#   },
#   {
#     "title": "Software Carpentry",
#     "location": "Virtual",
#     "date": "July 13-15, 2022",
#     "url": "https://www.raynamharris.com/2022-07-13-upr-online/"
#   },
#   {
#     "title": "Matplotlib Training",
#     "location": "Virtual",
#     "date": "April 21-22, 2022",
#     "url": "https://indico.cern.ch/event/1058838/"
#   },
#   {
#     "title": "Software Carpentry",
#     "location": "Virtual",
#     "date": "March 28-30, 2022",
#     "url": "https://indico.cern.ch/event/1112526/"
#   },
#   {
#     "title": "Data Analysis for Lab Research" ,
#     "location": "UPRM, Puerto Rico",
#     "date": "March 5, 2022",
#     "url": "https://indico.cern.ch/event/1132342/"
#   },
#   {
#     "title": "Software Carpentry",
#     "location": "Virtual",
#     "date": "Dec 13-15, 2021",
#     "url": "https://indico.cern.ch/event/1097111/"
#   },
#   {
#     "title": "HSF/IRISH-HEP Analysis Reproducibility",
#     "location": "Virtual",
#     "date": "Mar 24-28, 2025",
#     "url": "https://indico.cern.ch/event/1508102/timetable/"
#   },
#   {
#     "title": "",
#     "location": "",
#     "date": "",
#     "url": ""
#   },
#   {
#     "title": "",
#     "location": "",
#     "date": "",
#     "url": ""
#   }
# ]


## Creating the bibfile from inspire

In [ ]:
import requests

url = "https://inspirehep.net/api/literature?sort=mostrecent&size=25&page=1&q=a Guillermo.Fidalgo.1&format=bibtex"
inspire = requests.get(url).text
print(inspire)


# Not needed

# response = requests.get('https://inspirehep.net/api/literature?sort=mostrecent&size=25&page=1&q=Guillermo%20Fidalgo')
# url = response.json()['links']['bibtex']
# print(url)
# pprint.pprint(json_out['links'])
# inspire = requests.get(url).text

Making it into a script

In [ ]:
%%writefile createbib.py 
import requests

url = "https://inspirehep.net/api/literature?sort=mostrecent&size=25&page=1&q=a Guillermo.Fidalgo.1&format=bibtex"
inspire = requests.get(url).text

zenodo = r"""
@misc{alexander_moreno_briceno_2022_7115834,
  author       = {Alexander Moreno Briceño and Aman Goel and Guillermo Antonio Fidalgo Rodriguez},
  title        = {Teaching Python the Sustainable Way: Lessons Learned at HSF Training},
  month        = sep,
  year         = 2022,
  publisher    = {Zenodo},
  doi          = {10.5281/zenodo.7115834},
  url          = {https://doi.org/10.5281/zenodo.7115834}
}
"""
with open("bibfile.bib","w") as f:
    f.write(inspire)
    f.write(zenodo)

In [ ]:
! python createbib.py

## Making tests with Pandoc

Just run
```shell
pandoc -C _data/Publications.bib -t markdown_strict \
--csl american-physics-society.csl \
-o _includes/myPubs.md
```

In [ ]:
! pandoc -C bibfile.bib -t markdown_strict \
--csl american-physics-society.csl \
-o _includes/allPubs.md

In [ ]:
! pandoc -C bibfile.bib -t markdown_strict \
--csl american-physics-society.csl \
 -o _includes/deleteME.md

In [ ]:
%%writefile Fixbib.py

def fixbib(data):
    for id,line in enumerate(data):
        if "$" in line : 
            line = line.replace("$","$$").replace(r"\_","_")
            print(line)
            data[id] = line

with open("_includes/allPubs.md") as f:
    data = f.readlines()

fixbib(data)
with open("_includes/allPubs.md","w") as f:
    f.writelines(data)